## Value Iteration method for Frozen Lake

In [2]:
import gym
import collections
from tensorboardX import SummaryWriter

In [3]:
ENV_NAME = "FrozenLake-v0"
#ENV_NAME = "FrozenLake8x8-v0"      # uncomment for larger version
GAMMA = 0.9
TEST_EPISODES = 20

The
central data structures in this example are as follows:
- Reward table: A dictionary with the composite key "source state" + "action" +
"target state". The value is obtained from the immediate reward.
- Transitions table: A dictionary keeping counters of the experienced
transitions. The key is the composite "state" + "action", and the value is
another dictionary that maps the target state into a count of times that we
have seen it. For example, if in state 0 we execute action 1 ten times, after
three times it will lead us to state 4 and after seven times to state 5.
The entry with the key (0, 1) in this table will be a dict with contents
{4: 3, 5: 7}. We can use this table to estimate the probabilities of our
transitions.
- Value table: A dictionary that maps a state into the calculated value of this
state.

The overall logic of our code is simple: in the loop, we play 100 random steps from
the environment, populating the reward and transition tables. After those 100
steps, we perform a value iteration loop over all states, updating our value table.
Then we play several full episodes to check our improvements using the updated
value table. If the average reward for those test episodes is above the 0.8 boundary,
then we stop training. During the test episodes, we also update our reward and
transition tables to use all data from the environment.

We define the Agent class, which will keep our tables and contain functions
that we will be using in the training loop:

In [4]:
class Agent:
    def __init__(self):
        self.env = gym.make(ENV_NAME)
        self.state = self.env.reset()
        self.rewards = collections.defaultdict(float)
        self.transits = collections.defaultdict(
            collections.Counter)
        self.values = collections.defaultdict(float)

    def play_n_random_steps(self, count):
        for _ in range(count):
            action = self.env.action_space.sample()
            new_state, reward, is_done, _ = self.env.step(action)
            self.rewards[(self.state, action, new_state)] = reward
            self.transits[(self.state, action)][new_state] += 1
            self.state = self.env.reset() \
                if is_done else new_state

    def calc_action_value(self, state, action):
        target_counts = self.transits[(state, action)]
        total = sum(target_counts.values())
        action_value = 0.0
        for tgt_state, count in target_counts.items():
            reward = self.rewards[(state, action, tgt_state)]
            val = reward + GAMMA * self.values[tgt_state]
            action_value += (count / total) * val
        return action_value

    def select_action(self, state):
        best_action, best_value = None, None
        for action in range(self.env.action_space.n):
            action_value = self.calc_action_value(state, action)
            if best_value is None or best_value < action_value:
                best_value = action_value
                best_action = action
        return best_action

    def play_episode(self, env):
        total_reward = 0.0
        state = env.reset()
        while True:
            action = self.select_action(state)
            new_state, reward, is_done, _ = env.step(action)
            self.rewards[(state, action, new_state)] = reward
            self.transits[(state, action)][new_state] += 1
            total_reward += reward
            if is_done:
                break
            state = new_state
        return total_reward

    def value_iteration(self):
        for state in range(self.env.observation_space.n):
            state_values = [
                self.calc_action_value(state, action)
                for action in range(self.env.action_space.n)
            ]
            self.values[state] = max(state_values)

In the class constructor, we create the environment that we will be using for data
samples, obtain our first observation, and define tables for rewards, transitions,
and values. We use the play_n_random_steps function is used to gather random experience from the environment and update
the reward and transition tables. Note that we don't need to wait for the end of the
episode to start learning; we just perform N steps and remember their outcomes.
This is one of the differences between value iteration and the cross-entropy method,
which can learn only on full episodes.

The function **calc_action_value** calculates the value of the action from the state using our
transition, reward, and values tables. We will use it for two purposes: to select the
best action to perform from the state and to calculate the new value of the state on
value iteration. We do the following:
1. We extract transition counters for the given state and action from the
transition table. Counters in this table have a form of dict, with target states
as the key and a count of experienced transitions as the value. We sum all
counters to obtain the total count of times we have executed the action from
the state. We will use this total value later to go from an individual counter
to probability.
2. Then we iterate every target state that our action has landed on and calculate
its contribution to the total action value using the Bellman equation. This
contribution is equal to immediate reward plus discounted value for the
target state. We multiply this sum to the probability of this transition and
add the result to the final action value.

Then, the approximate value for the state and action, Q(s, a), will be equal to the
probability of every state, multiplied to the value of the state. From the Bellman
equation, this equals the sum of the immediate reward and the discounted long-term
state value.

**The select_action function uses the function that I just described to make a decision about
the best action to take from the given state. It iterates over all possible actions in
the environment and calculates the value for every action. The action with the
largest value wins and is returned as the action to take. This action selection process
is deterministic, as the play_n_random_steps() function introduces enough
exploration. So, our agent will behave greedily in regard to our value approximation.**

**The play_episode() function uses select_action() to find the best action to take
and plays one full episode using the provided environment. This function is used to
play test episodes, during which we don't want to mess with the current state of the
main environment used to gather random data. So, we use the second environment
passed as an argument. The logic is very simple and should be already familiar to
you: we just loop over states accumulating reward for one episode.**

**The final method of the Agent class is our value iteration implementation and it
is surprisingly simple, thanks to the preceding functions. What we do is just loop
over all states in the environment, then for every state, we calculate the values for
the states reachable from it, obtaining candidates for the value of the state. Then we
update the value of our current state with the maximum value of the action available
from the state.**

In [5]:
test_env = gym.make(ENV_NAME)
agent = Agent()
writer = SummaryWriter(comment="-v-iteration")

iter_no = 0
best_reward = 0.0
while True:
    iter_no += 1
    agent.play_n_random_steps(100)
    agent.value_iteration()

    reward = 0.0
    for _ in range(TEST_EPISODES):
        reward += agent.play_episode(test_env)
    reward /= TEST_EPISODES
    writer.add_scalar("reward", reward, iter_no)
    if reward > best_reward:
        print("Best reward updated %.3f -> %.3f" % (
            best_reward, reward))
        best_reward = reward
    if reward > 0.80:
        print("Solved in %d iterations!" % iter_no)
        break
writer.close()

Best reward updated 0.000 -> 0.250
Best reward updated 0.250 -> 0.350
Best reward updated 0.350 -> 0.400
Best reward updated 0.400 -> 0.500
Best reward updated 0.500 -> 0.550
Best reward updated 0.550 -> 0.600
Best reward updated 0.600 -> 0.650
Best reward updated 0.650 -> 0.700
Best reward updated 0.700 -> 0.750
Best reward updated 0.750 -> 0.800
Best reward updated 0.800 -> 0.850
Solved in 88 iterations!


First, we perform 100 random steps to fill our reward and transition tables with
fresh data, and then we run value iteration over all states. The rest of the code plays
test episodes using the value table as our policy, then writes data into TensorBoard,
tracks the best average reward, and checks for the training loop stop condition.

Our solution is stochastic, and my experiments usually required from 12 to 100
iterations to reach a solution, but in all cases, it took less than a second to find a good
policy that could solve the environment in 80% of runs. If you remember how many
hours were required to achieve a 60% success ratio using the cross-entropy method,
then you can understand that this is a major improvement. There are several reasons
for that.

First of all, the stochastic outcome of our actions, plus the length of the episodes
(six to 10 steps on average), makes it hard for the cross-entropy method to
understand what was done right in the episode and which step was a mistake. Value
iteration works with individual values of the state (or action) and incorporates the
probabilistic outcome of actions naturally by estimating probability and calculating
the expected value. So, it's much simpler for value iteration and requires much less
data from the environment (which is called sample efficiency in RL).

The second reason is the fact that value iteration doesn't need full episodes to
start learning. In an extreme case, we can start updating our values just from a
single example. However, for FrozenLake, due to the reward structure (we get
1 only after successfully reaching the target state), we still need to have at least
one successful episode to start learning from a useful value table, which may
be challenging to achieve in more complex environments. For example, you can
try switching the existing code to a larger version of FrozenLake, which has the
name FrozenLake8x8-v0. The larger version of FrozenLake can take from 150 to
1,000 iterations to solve, and, according to TensorBoard charts, most of the time
it waits for the first successful episode, then it very quickly reaches convergence.